It happens all the time: someone gives you data containing malformed strings, Python, lists and missing data. How do you tidy it up so you can get on with the analysis?  Take this monstrosity as the DataFrame to use in the following puzzles:

In [371]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm',  'Budapest_PaRis', 'Brussels_londOn'],  'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],  'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],  'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',  '12. Air France', '"Swiss Air"']}) 
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


1. Some values in the the FlightNumber column are missing. These numbers are meant to increase by 10 with each row so 10055 and 10075 need to be put in place. Fill in these missing numbers and make the column an integer column (instead of a float column). 

In [372]:
i_previsousdataval = 0
for i in range(0,len(df['FlightNumber']-1)): 
    if df['FlightNumber'].isnull()[i]:
        df['FlightNumber'][i] = i_previsousdataval + 10
    i_previsousdataval = df['FlightNumber'][i]

C:\Users\Siddharth\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [373]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055.0,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075.0,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


In [374]:
df['FlightNumber'].dtype

dtype('float64')

In [375]:
df['FlightNumber'] = df['FlightNumber'].astype(np.int64)
df['FlightNumber'].dtype

dtype('int64')

2. The From_To column would be better as two separate columns! Split each string on the underscore delimiter _ to give a new temporary DataFrame with the correct values. Assign the correct column names to this temporary DataFrame. 

In [376]:
dftemp = df.copy()

In [377]:
dftemp['From'] = df['From_To'].str.split('_').str[0]
dftemp['To'] = df['From_To'].str.split('_').str[1]

In [378]:
dftemp

,From_To,FlightNumber,RecentDelays,Airline,From,To
0,LoNDon_paris,10045,"[23, 47]",KLM(!),LoNDon,paris
1,MAdrid_miLAN,10055,[],<Air France> (12),MAdrid,miLAN
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. ),londON,StockhOlm
3,Budapest_PaRis,10075,[13],12. Air France,Budapest,PaRis
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air""",Brussels,londOn


3. Notice how the capitalisation of the city names is all mixed up in this temporary DataFrame. Standardise the strings so that only the first letter is uppercase (e.g. "londON" should become "London".) 

In [379]:
dftemp['From'] = dftemp['From'].str.capitalize()
dftemp['To'] = dftemp['To'].str.capitalize()

In [380]:
dftemp

,From_To,FlightNumber,RecentDelays,Airline,From,To
0,LoNDon_paris,10045,"[23, 47]",KLM(!),London,Paris
1,MAdrid_miLAN,10055,[],<Air France> (12),Madrid,Milan
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,Budapest_PaRis,10075,[13],12. Air France,Budapest,Paris
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air""",Brussels,London


4. Delete the From_To column from df and attach the temporary DataFrame from the previous questions.

In [381]:
df.drop(['From_To'], axis=1, inplace=True)

In [382]:
df[dftemp[['From','To']].columns] = dftemp[['From','To']]

In [383]:
df

,FlightNumber,RecentDelays,Airline,From,To
0,10045,"[23, 47]",KLM(!),London,Paris
1,10055,[],<Air France> (12),Madrid,Milan
2,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,10075,[13],12. Air France,Budapest,Paris
4,10085,"[67, 32]","""Swiss Air""",Brussels,London


5. In the RecentDelays column, the values have been entered into the DataFrame as a list. We would like each first value in its own column, each 
second value in its own column, and so on. If there isn't an Nth value, the value should be NaN. 
Expand the Series of lists into a DataFrame named delays, rename the columns delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df with delays. 

In [393]:
delays = pd.DataFrame(df['RecentDelays'].to_list())
delays

,0,1,2
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [394]:
for col in delays.columns: 
    colname = 'delay_' + str(col+1) 
    delays.rename(columns={col:colname}, inplace=True)
delays

,delay_1,delay_2,delay_3
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [395]:
rd_idx = df.columns.get_loc('RecentDelays') 

for col in range(rd_idx,len(delays.columns)+1):
    colnm = 'delay_' + str(col)
    df.insert(loc = col , column=colnm , value=delays[colnm])
df.drop(columns="RecentDelays",inplace=True)
df

,FlightNumber,delay_1,delay_2,delay_3,Airline,From,To
0,10045,23.0,47.0,NaN,KLM(!),London,Paris
1,10055,NaN,NaN,NaN,<Air France> (12),Madrid,Milan
2,10065,24.0,43.0,87.0,(British Airways. ),London,Stockholm
3,10075,13.0,NaN,NaN,12. Air France,Budapest,Paris
4,10085,67.0,32.0,NaN,"""Swiss Air""",Brussels,London
